# Initializing Neural Networks

In this notebook we discuss how to initialize neural networks.  We'll need numpy:

In [1]:
import numpy as np

## Trouble with Symmetry

Recall that the first layer has `n` inputs, the last layer has `k` outputs, and we have a bunch of natural numbers (well, possibly zero, actually) to determine the input sizes of all the intermediate layers.  That, along with the chosen activation functions, determines the layout of the network.  Therefore, we might expect our initialization functions to look like this:

In [2]:
def initial_network(n, k, intermediate_sizes):
    dimensions = [n] + intermediate_sizes + [k] # input/output sizes
    weights = [0] * (len(dimensions)-1) # the neurons themselves
    biases = [0] * (len(dimensions)-1)
    
    for i in range(0, len(weights)):
        weights[i] = np.zeros((dimensions[i], dimensions[i+1]))
        biases[i] = np.zeros((dimensions[i+1], 1))
    
    return weights, biases

Indeed this generates matrices of the appropriate size, but there is a problem. It turns out that if two neurons are literally equal, they will never be able to separate.  They will face the same selection pressures and train the same way, so will stay equal forever.

We need to solve this problem, and any solution to it is called *symmetry breaking* in the literature.

## Basic Randomization

A common way to do this, and it isn't too bad, is to just make everything random. For example, we might literally just make everything come from uniform random noise, between -1 and 1:

In [3]:
def initialize_network_uniform(n, k, intermediate_sizes):
    dimensions = [n] + intermediate_sizes + [k] # input/output sizes
    weights = [0] * (len(dimensions)-1) # the neurons themselves
    biases = [0] * (len(dimensions)-1)
    
    for i in range(0, len(weights)):
        weights[i] = np.random.random((dimensions[i], dimensions[i+1]))*2 - 1
        biases[i] = np.random.random((1, dimensions[i+1]))*2 - 1
    
    return weights, biases

Recall that `np.random.random()` is a random float between 0 and 1, so `np.random.random()*2-1` is a random float between -1 and 1.

## More Advanced Randomization

Some amount of research has gone into this, though, and it seems wasteful not to take advantage of it. According to Yoshua Bengio, one should still use uniform random numbers for each matrix, but with the following conditions:
1. Bias units and output units should be initialized to zero; training will sort them out.
2. Sigmoid units should use uniform random weights from $-r$ to $r$, where $r=4\sqrt{6/(\textrm{fan-in}+\textrm{fan-out})}$.
3. Hyperbolic tangent units should use uniform random weights from $-r$ to $r$, where $r=\sqrt{6/(\textrm{fan-in}+\textrm{fan-out})}$.
4. LeRU units should use Gaussian-distributed random weights with standard deviation $r=\sqrt{2/(\textrm{fan-in})}$.

Here fan-in is the number of inputs to the matrix, and fan-out is the number of outputs of the matrix.  Basically this is just fine-tuning the idea that we should have lower weights when more neurons are feeding it, and lower weights when more neurons are looking at you.  This is all to prevent saturation.

If you're interested, there's a more in-depth explanation at <a href="http://andyljones.tumblr.com/post/110998971763/an-explanation-of-xavier-initialization">andy's blog</a>.  The original paper (by Xaviar Glorot and Yoshua Bengio; the algorithm is named for the first author) is <a href="http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf">here</a>, and they work out the details for sigmoid and hyperbolic tangent units.  The formula for rectifier units is worked out <a href="https://arxiv.org/pdf/1502.01852v1.pdf">here</a>.

It seems to me from reading that what's important is that the mean be zero, and the variance be appropriately defined from the fan-in and fan-out.  The choice of uniform or Gaussian random numbers seems to be the preference of the authors, but I'm not going to override them.  They did a lot of experimental work to get these figures!

With all that in mind, we give the following initialization process:

In [4]:
def initialize_xavier_sigmoid(n, k, intermediate_sizes):
    dimensions = [n] + intermediate_sizes + [k] # input/output sizes
    weights = [0] * (len(dimensions)-1) # the neurons themselves
    biases = [0] * (len(dimensions)-1)
    
    for i in range(0, len(weights)-1):
        r = 4 * ((6/(dimensions[i]+dimensions[i+1]))**0.5)
        weights[i] = np.random.random((dimensions[i], dimensions[i+1]))*(2*r) - r
        biases[i] = np.zeros((1, dimensions[i+1]))
    
    # set the last ones to zero
    weights[-1] = np.zeros((dimensions[-2], dimensions[-1]))
    biases[-1] = np.zeros((1, dimensions[-1]))
    
    return weights, biases

def initialize_xavier_tanh(n, k, intermediate_sizes):
    dimensions = [n] + intermediate_sizes + [k] # input/output sizes
    weights = [0] * (len(dimensions)-1) # the neurons themselves
    biases = [0] * (len(dimensions)-1)
    
    for i in range(0, len(weights)-1):
        r = ((6/(dimensions[i]+dimensions[i+1]))**0.5)
        weights[i] = np.random.random((dimensions[i], dimensions[i+1]))*(2*r) - r
        biases[i] = np.zeros((1, dimensions[i+1]))
    
    # set the last ones to zero
    weights[-1] = np.zeros((dimensions[-2], dimensions[-1]))
    biases[-1] = np.zeros((1, dimensions[-1]))
    
    return weights, biases

def initialize_xavier_leru(n, k, intermediate_sizes):
    dimensions = [n] + intermediate_sizes + [k] # input/output sizes
    weights = [0] * (len(dimensions)-1) # the neurons themselves
    biases = [0] * (len(dimensions)-1)
    
    for i in range(0, len(weights)-1):
        r = (2/(dimensions[i]))**0.5
        weights[i] = np.random.standard_normal((dimensions[i], dimensions[i+1]))*r
        biases[i] = np.zeros((1, dimensions[i+1]))
    
    # set the last ones to zero
    weights[-1] = np.zeros((dimensions[-2], dimensions[-1]))
    biases[-1] = np.zeros((1, dimensions[-1]))
    
    return weights, biases

Once we get our learning algorithm up and running, we'll experiment with the effect of these initialization methods.